# Setup
This IPython notebook will walk through the steps of characterizing iModulons through the semi-automated tools in PyModulon. You will need:

* M and A matrices
* Expression data (e.g. `log_tpm_norm.csv`)
* Gene table and KEGG/GO annotations (Generated in `gene_annotation.ipynb`)
* Sample table, with a column for `project` and `condition`
* TRN file

Optional:
* iModulon table (if you already have some characterized iModulons)

In [1]:
from pymodulon.core import IcaData
from pymodulon.plotting import *
from os import path
import pandas as pd
import re
from Bio.KEGG import REST
from tqdm.notebook import tqdm

In [2]:
# Enter the location of your data here
data_dir = path.join('/Users/louxuwen/Desktop/Documents/GitHub/modulome-C_Glutamicum_Microarray_clean/7_characterizing_imodulons/Data')

# GO and KEGG annotations are in the 'external' folder
# external_data = path.join('..','data','external')

## Check your sample table (i.e. metadata file)
Your metadata file will probably have a lot of columns, most of which you may not care about. Feel free to save a secondary copy of your metadata file with only columns that seem relevant to you. The two most important columns are:
1. `project`
2. `condition`

Make sure that these columns exist in your metadata file

In [3]:
df_metadata = pd.read_csv(path.join(data_dir,'metadata.tsv'),index_col=0,sep='\t')
df_metadata[['project','condition']].head()

,project,condition
Assigned ID,,
GSM5197020_1,P1,Condition526
GSM5197020_2,P1,Condition126
GSM5197021_1,P1,Condition526
GSM5197021_2,P1,Condition126
GSM5197022_1,P1,Condition526


In [4]:
print(df_metadata.project.notnull().all())
print(df_metadata.condition.notnull().all())

True
True


## Check your TRN

Each row of the TRN file represents a regulatory interaction.  
**Your TRN file must have the following columns:**
1. `regulator` - Name of regulator (`/` or `+` characters will be converted to `;`)
1. `gene_id` - Locus tag of gene being regulated

The following columns are optional, but are helpful to have:
1. `regulator_id` - Locus tag of regulator
1. `gene_name` - Name of gene (can automatically update this using `name2num`)
1. `direction` - Direction of regulation ('+' for activation, '-' for repression, '?' or NaN for unknown)
1. `evidence` - Evidence of regulation (e.g. ChIP-exo, qRT-PCR, SELEX, Motif search)
1. `PMID` - Reference for regulation

You may add any other columns that could help you. TRNs may be saved as either CSV or TSV files. See below for an example:

In [ ]:
df_trn = pd.read_csv(path.join(external_data,'TRN.csv'))
df_trn.head()

The `regulator` and `gene_id` must be filled in for each row

In [ ]:
print(df_trn.regulator.notnull().all())
print(df_trn.gene_id.notnull().all())

## Load the data
You're now ready to load your IcaData object!

In [ ]:
#ica_data = IcaData(M = path.join(data_dir,'M.csv'),
#                   A = path.join(data_dir,'A.csv'),
#                   X = path.join(data_dir,'log_tpm_norm.csv'),
#                   gene_table = path.join(data_dir,'gene_info.csv'),
#                   sample_table = path.join(data_dir,'metadata.tsv'),
#                   #trn = path.join(external_data,'TRN.csv'),
#                   optimize_cutoff=True)

If you don't have a TRN (or have a very minimal TRN), use `threshold_method = 'kmeans'`

In [5]:
ica_data = IcaData(M = path.join(data_dir,'M.csv'),
                   A = path.join(data_dir,'A.csv'),
                   X = path.join(data_dir,'log_tpm_norm.csv'),
                   gene_table = path.join(data_dir,'gene_info.csv'),
                   sample_table = path.join(data_dir,'metadata.tsv'),
                   #trn = path.join(data_dir,'TRN.csv'),
                   threshold_method = 'kmeans')

# Regulatory iModulons
Use `compute_trn_enrichment` to automatically check for Regulatory iModulons. The more complete your TRN, the more regulatory iModulons you'll find.

In [7]:
#ica_data.compute_trn_enrichment()

You can also search for AND/OR combinations of regulators using the `max_regs` argument.

Regulator enrichments can be directly saved to the `imodulon_table` using the `save` argument. This saves the enrichment with the lowest q-value to the table.

In [ ]:
# First search for regulator enrichments with 2 regulators
ica_data.compute_trn_enrichment(max_regs=2,save=True)

# Next, search for regulator enrichments with just one regulator. This will supercede the 2 regulator enrichments.
ica_data.compute_trn_enrichment(max_regs=1,save=True)

The list of regulatory iModulons are shown below

In [ ]:
regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]
print(len(ica_data.imodulon_table),'Total iModulons')
print(len(regulatory_imodulons),'Regulatory iModulons')
regulatory_imodulons

You can rename iModulons in this jupyter notebook, or you can save the iModulon table as a CSV and edit it in Excel.

If two iModulons have the same regulator (e.g. 'Reg'), they will be named 'Reg-1' and 'Reg-2'

In [ ]:
ica_data.rename_imodulons(regulatory_imodulons.regulator.to_dict())
ica_data.imodulon_table.head()

In [ ]:
regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]

# Functional iModulons

GO annotations and KEGG pathways/modules were generated in the 1_create_the_gene_table.ipynb notebook. Enrichments will be calculated in this notebook, and further curated in the 3_manual_iModulon_curation notebook.

## GO Enrichments

First load the Gene Ontology annotations

In [ ]:
DF_GO = pd.read_csv(path.join(external_data,'GO_annotations_curated.csv'),index_col=0)
DF_GO.head()

In [ ]:
DF_GO_enrich = ica_data.compute_annotation_enrichment(DF_GO,'gene_ontology')

In [ ]:
DF_GO_enrich.head()

## KEGG Enrichments

### Load KEGG mapping
The `kegg_mapping.csv` file contains KEGG orthologies, pathways, modules, and reactions. Only pathways and modules are relevant to iModulon characterization.

In [9]:
DF_KEGG = pd.read_csv(path.join(data_dir,'kegg_mapping.csv'),index_col=0)
print(DF_KEGG.database.unique())
DF_KEGG.head()

['KEGG_orth' 'KEGG_pathway' 'KEGG_module' 'KEGG_reaction']


,gene_id,database,kegg_id
3,cg0006,KEGG_orth,-
5,cg0008,KEGG_orth,-
7,cg0010,KEGG_orth,-
8,cg0012,KEGG_orth,-
9,cg0013,KEGG_orth,-


In [10]:
kegg_pathways = DF_KEGG[DF_KEGG.database == 'KEGG_pathway']
kegg_modules = DF_KEGG[DF_KEGG.database == 'KEGG_module']

### Perform enrichment
Uses the `compute_annotation_enrichment` function

In [11]:
DF_pathway_enrich = ica_data.compute_annotation_enrichment(kegg_pathways,'kegg_id')
DF_module_enrich = ica_data.compute_annotation_enrichment(kegg_modules,'kegg_id')

In [12]:
DF_pathway_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,4,map00920,1.110789e-10,2.521490e-08,0.714286,0.263158,0.384615,5.0,19.0,7.0
1,5,map02024,1.620805e-05,3.679227e-03,0.238095,0.100000,0.140845,5.0,50.0,21.0
2,14,map00190,4.170788e-09,9.467689e-07,1.000000,0.153846,0.266667,4.0,26.0,4.0
3,19,map00906,4.046112e-04,9.184674e-02,0.100000,0.400000,0.160000,2.0,5.0,20.0
4,24,map01230,1.601619e-05,3.635675e-03,0.304348,0.059322,0.099291,7.0,118.0,23.0


In [13]:
DF_module_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,4,M00436,7.430699e-09,0.000002,0.428571,1.000000,0.600000,3.0,3.0,7.0
1,4,M00188,4.510449e-05,0.005255,0.285714,0.400000,0.333333,2.0,5.0,7.0
2,5,M00239,6.051004e-06,0.001410,0.190476,0.210526,0.200000,4.0,19.0,21.0
3,5,M00125,6.675746e-04,0.077772,0.095238,0.333333,0.148148,2.0,6.0,21.0
4,11,M00179,2.987003e-04,0.069597,0.666667,0.064516,0.117647,2.0,31.0,3.0


### Convert KEGG IDs to human-readable names

In [14]:
for idx,key in tqdm(DF_pathway_enrich.kegg_id.items(),total=len(DF_pathway_enrich)):
    text = REST.kegg_find('pathway',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_pathway_enrich.loc[idx,'pathway_name'] = name
    except AttributeError:
        DF_pathway_enrich.loc[idx,'pathway_name'] = None
    
for idx,key in tqdm(DF_module_enrich.kegg_id.items(),total=len(DF_module_enrich)):
    text = REST.kegg_find('module',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_module_enrich.loc[idx,'module_name'] = name
    except AttributeError:
        DF_module_enrich.loc[idx,'module_name'] = None

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

In [15]:
DF_pathway_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,pathway_name
0,4,map00920,1.110789e-10,2.521490e-08,0.714286,0.263158,0.384615,5.0,19.0,7.0,Sulfur metabolism
1,5,map02024,1.620805e-05,3.679227e-03,0.238095,0.100000,0.140845,5.0,50.0,21.0,Quorum sensing
2,14,map00190,4.170788e-09,9.467689e-07,1.000000,0.153846,0.266667,4.0,26.0,4.0,Oxidative phosphorylation
3,19,map00906,4.046112e-04,9.184674e-02,0.100000,0.400000,0.160000,2.0,5.0,20.0,Carotenoid biosynthesis
4,24,map01230,1.601619e-05,3.635675e-03,0.304348,0.059322,0.099291,7.0,118.0,23.0,Biosynthesis of amino acids


In [16]:
DF_module_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,module_name
0,4,M00436,7.430699e-09,0.000002,0.428571,1.000000,0.600000,3.0,3.0,7.0,NaN
1,4,M00188,4.510449e-05,0.005255,0.285714,0.400000,0.333333,2.0,5.0,7.0,NaN
2,5,M00239,6.051004e-06,0.001410,0.190476,0.210526,0.200000,4.0,19.0,21.0,NaN
3,5,M00125,6.675746e-04,0.077772,0.095238,0.333333,0.148148,2.0,6.0,21.0,"Riboflavin biosynthesis, plants and bacteria, ..."
4,11,M00179,2.987003e-04,0.069597,0.666667,0.064516,0.117647,2.0,31.0,3.0,None


## SubtiWiki categories

In [ ]:
DF_subtiwiki = pd.read_csv(path.join(external_data,'subtiwiki_categories.csv'))
DF_subtiwiki.head()

In [ ]:
# Change the subtiwiki annotation format into a list of genes and categories
DF_subtiwiki = DF_subtiwiki.rename({'BSU_number':'gene_id'},axis=1)
DF_subtiwiki = DF_subtiwiki.melt(id_vars='gene_id',value_vars=['FuncName1','FuncName2','FuncName3','FuncName4','FuncName5'])
DF_subtiwiki = DF_subtiwiki[DF_subtiwiki.value.notnull() & DF_subtiwiki.gene_id.isin(ica_data.gene_names)]
DF_subtiwiki.head()

In [ ]:
DF_subti_enrich = ica_data.compute_annotation_enrichment(DF_subtiwiki,'value')

## Save files

In [17]:
# DF_GO_enrich['source'] = 'GO'
DF_pathway_enrich['source'] = 'KEGG pathways'
DF_module_enrich['source'] = 'KEGG modules'
# DF_subti_enrich['source'] = 'SubtiWiki'

# DF_GO_enrich.rename({'gene_ontology':'annotation'},axis=1, inplace=True)
DF_pathway_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
DF_module_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
# DF_subti_enrich.rename({'value':'annotation'},axis=1, inplace=True)

DF_enrichments = pd.concat([DF_pathway_enrich, DF_module_enrich])
DF_enrichments.to_csv(path.join(data_dir,'functional_enrichments.csv'))

# Check for single gene iModulons

Some iModulons are dominated by a single, high-coefficient gene. These iModulons may result from:
1. Overdecomposition of the dataset to identify noisy genes
1. Artificial knock-out of single genes
1. Regulons with only one target gene

No matter what causes these iModulons, it is important to be aware of them. The find_single_gene_imodulons function identifies iModulons that are likely dominated by a single gene.

The iModulons identified by ``find_single_gene_imodulons`` may contain more than one gene, since a threshold-agnostic method is used to identify these iModulons.

In [18]:
sg_imods = ica_data.find_single_gene_imodulons(save=True)
len(sg_imods)

44

In [19]:
for i,mod in enumerate(sg_imods):
    ica_data.rename_imodulons({mod:'SG_'+str(i+1)})

In [20]:
ica_data.imodulon_table[ica_data.imodulon_table.single_gene == True]

,single_gene
SG_1,True
SG_2,True
SG_3,True
SG_4,True
SG_5,True
SG_6,True
SG_7,True
SG_8,True
SG_9,True
SG_10,True


In [21]:
ica_data.view_imodulon('SG_1')

,gene_weight,gene_name,accession,old_locus_tag,start,end,strand,gene_product,COG,uniprot,operon
cg1514,0.925392,cg1514,BX927147.1,NaN,1406784.0,1407536.0,-,secreted protein,No COG annotation,NaN,Op857


# Save iModulon object

In [22]:
from pymodulon.util import explained_variance
from pymodulon.io import *

In [23]:
# Add iModulon sizes and explained variance
for im in ica_data.imodulon_names:
    ica_data.imodulon_table.loc[im,'imodulon_size'] = len(ica_data.view_imodulon(im))
    ica_data.imodulon_table.loc[im,'explained_variance'] = explained_variance(ica_data,imodulons=im)

This will save your iModulon table, your thresholds, and any other information stored in the ica_data object.

In [24]:
save_to_json(ica_data, path.join(data_dir,'bsu_raw.json.gz'))

If you prefer to view and edit your iModulon table in excel, save it as a CSV and reload the iModulon as before

In [25]:
ica_data.imodulon_table.to_csv(path.join(data_dir,'imodulon_table_raw.csv'))